In [132]:
%load_ext autoreload
%autoreload 2

In [131]:
from __future__ import print_function, absolute_import

import numpy as np
import matplotlib.pyplot as plt
import unittest
import warnings

import pydrake
from pydrake.all import (
    SolutionResult
)
import pydrake.symbolic as sym
from pydrake.solvers import mathematicalprogram as mp
from pydrake.solvers.mathematicalprogram import SolverType
import pydrake.autodiffutils
from pydrake.autodiffutils import AutoDiffXd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


prog = mp.MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")
# prog.AddLinearConstraint(x[0] >= 1)
# prog.AddLinearConstraint(x[1] >= 1)
# prog.AddQuadraticCost(np.eye(2), np.zeros(2), x)
# Redundant cost just to check the spelling.
# prog.AddQuadraticErrorCost(vars=x, Q=np.eye(2),
#                            x_desired=np.zeros(2))
# prog.AddL2NormCost(A=np.eye(2), b=np.zeros(2), vars=x)

class FC(nn.Module):
    def __init__(self, layer_norm=False):
        super(FC, self).__init__()
        self.fc1 = nn.Linear(2, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

class MLP(nn.Module):
    def __init__(self, layer_norm=False):
        super(MLP, self).__init__()
        self.layer_norm = layer_norm

        self.l1 = nn.Linear(2, 64)
        self.ln1 = nn.LayerNorm(64)
        self.tanh1 = F.tanh
        self.l2 = nn.Linear(64, 64)
        self.ln2 = nn.LayerNorm(64)
        self.tanh2 = F.tanh
        self.l3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.l1(x)
        if self.layer_norm: x = self.ln1(x)
        x = self.tanh1(x)
        x = self.l2(x)
        if self.layer_norm: x = self.ln2(x)
        x = self.tanh2(x)
        x = self.l3(x)
        return x
    
# net = FC()
net = MLP()

# This function uses an neural network, but only has one output...
def autodiffable_function_nn(drake_in):
    global net

    # Convert input to a torch tensor
    print("drake_in: ", drake_in)
    n_inputs = drake_in.shape[0]
    just_values = np.array([drake_in[i].value() for i in range(drake_in.shape[0])])
    torch_in = torch.tensor(just_values, requires_grad=True)
    print("torch_in: ", torch_in)
                                                                                                                                                                                                                                                       
    # Run the forward pass.
    # We'll do the backward pass(es) when we calculate output and it's gradients.
    torch_out = net.forward(torch_in)
    print("torch_out: ", torch_out)
    
    # Currently we only support one output
    assert torch_out.shape[0] == 1, "Need just one output for valid cost function"
    
    # Do derivative calculation and pack into the output vector.
    # Because neural network might have multiple outputs, I can't simply use net.backward() with no argument.
    # Instead need to follow the advice here:
    #     https://discuss.pytorch.org/t/clarification-using-backward-on-non-scalars/1059
    n_outputs = torch_out.shape[0]
    drake_out = [0]*n_outputs
    for j in range(n_outputs):
        print("\niter j: ", j)
        y_j_value = torch_out[j].clone().detach().numpy()
        # Equation: y.derivs = dydu*u.derivs() + dydp*p.derivs()
        # Alternate equation, for each y, y_j.deriv = sum_i  (dy_jdu[i] * u[i].deriv)
        #                          (#y's) (1x#derivs) (#u's)  (1x#u's)    (1x#derivs)

        # Make empty accumulator
        y_j_deriv = np.zeros_like(drake_in[0].derivatives())
        
        # https://discuss.pytorch.org/t/clarification-using-backward-on-non-scalars/1059
        output_selector = torch.zeros(1, n_outputs)
        output_selector[j] = 1.0 # Set the output we want a derivative w.r.t. to 1.
        torch_out.backward(output_selector, retain_graph=True)
        dy_jdu = torch_in.grad.numpy() # From Torch, give us back a numpy object
        for i in range(n_inputs):
            u_i_deriv = drake_in[i].derivatives()
            print("dy_jdu_a[i] * u_i_deriv = ", dy_jdu[i], " * ",  u_i_deriv)
            y_j_deriv += dy_jdu[i] * u_i_deriv;
        print("putting into output: ", y_j_value, ", ", y_j_deriv)
        drake_out[j] = AutoDiffXd(y_j_value, y_j_deriv)
    return drake_out[0]

def autodiffable_function_nn(drake_in):
    

def autodiffable_function_simple(x):
    print(type(x), x.shape)
    print(type(x[0]), x.shape)
    return (x[0]-2.)*(x[1]-2.)

# prog.AddCost(autodiffable_function_simple, x)
prog.AddCost(autodiffable_function_nn, x)

result = prog.Solve()
print(result, mp.SolutionResult.kSolutionFound)

x_expected = np.array([1, 1])
print(np.allclose(prog.GetSolution(x), x_expected))

print(prog.GetSolution(x))

drake_in:  [<AutoDiffXd 0.0 nderiv=2> <AutoDiffXd 0.0 nderiv=2>]
torch_in:  tensor([0., 0.], requires_grad=True)
torch_out:  tensor([-0.2056], grad_fn=<AddBackward0>)

iter j:  0
dy_jdu_a[i] * u_i_deriv =  0.07312082604007633  *  [1. 0.]
dy_jdu_a[i] * u_i_deriv =  -0.13104201982836147  *  [0. 1.]
putting into output:  -0.20559247671308717 ,  [ 0.07312083 -0.13104202]
drake_in:  [<AutoDiffXd 2.73871029615e-07 nderiv=2>
 <AutoDiffXd -2.73868290904e-07 nderiv=2>]
torch_in:  tensor([ 2.7387e-07, -2.7387e-07], requires_grad=True)
torch_out:  tensor([-0.2056], grad_fn=<AddBackward0>)

iter j:  0
dy_jdu_a[i] * u_i_deriv =  0.0731208355216025  *  [1. 0.]
dy_jdu_a[i] * u_i_deriv =  -0.1310420400494838  *  [0. 1.]
putting into output:  -0.20559242079915319 ,  [ 0.07312084 -0.13104204]
drake_in:  [<AutoDiffXd -0.0638964231999 nderiv=2>
 <AutoDiffXd 0.114510691541 nderiv=2>]
torch_in:  tensor([-0.0639,  0.1145], requires_grad=True)
torch_out:  tensor([-0.2247], grad_fn=<AddBackward0>)

iter j:  0


In [ ]:
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.autodiffutils import AutoDiffXd


In [35]:
# Set Up Mathematical Program
prog = mp.MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")
z = prog.NewContinuousVariables(1, "z")
# print(type(x[0]), type(z[0]))
prog.AddCost(z[0])

# Add LorentzConeConstraints
prog.AddLorentzConeConstraint(np.array([0*x[0]+1, x[0]-1, x[1]-1]))
prog.AddLorentzConeConstraint(np.array([z[0], x[0], x[1]]))

def autodiffable_function(x):
    print(type(x[0]))
    print(x[0].value())
    print(x[0].derivatives())
    return (x[0]-2.)*(x[1]-2.)
prog.AddCost(autodiffable_function, x)

# Test result
result = prog.Solve()
print(result, mp.SolutionResult.kSolutionFound)

# Check answer
x_expected = np.array([1-2**(-0.5), 1-2**(-0.5)])
print(np.allclose(prog.GetSolution(x), x_expected))

print(prog.GetSolution(x), prog.GetSolution(z))

<class 'pydrake.autodiffutils.AutoDiffXd'>
0.0
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
1.8258068641e-07
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.514568548895
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.257284274447
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.488616822259
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.255438203943
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.292010091187
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
-2.29201009119
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.291948908967
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.292892542544
[1. 0.]
<class 'pydrake.autodiffutils.AutoDiffXd'>
0.292892542544
[1. 0.]
(SolutionResult.kInfeasibleConstraints, SolutionResult.kSolutionFound)
True
(array([0.29289254, 0.29289264]), array([0.]))


In [74]:
torch.set_default_dtype(torch.float64)
x = torch.tensor([[2,1]], dtype=torch.float32, requires_grad=True)
M = torch.tensor([[1,2],[3,4]], dtype=torch.float32)
y = torch.mm(x, M)
jacobian = torch.zeros((2, 2))
y.backward(torch.tensor([[1., 0.]], dtype=torch.float32), retain_graph=True)
jacobian[:,0] = x.grad.data
x.grad.data.zero_()
y.backward(torch.tensor([[0, 1]], dtype=torch.float32), retain_graph=True)
jacobian[:,1] = x.grad.data

In [75]:
jacobian

tensor([[1., 2.],
        [3., 4.]])

In [72]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
out = x.pow(2).sum()
out.backward()
x.grad

tensor([[ 2., -2.],
        [ 2.,  2.]])

In [128]:
autodiff_in = np.array([
    [AutoDiffXd(1.0, [1.0, 0.0]),],
    [AutoDiffXd(1.0, [1.0, 0.0]),]
])
autodiff_in
# type(autodiff_in), autodiff_in.shape

array([[<AutoDiffXd 1.0 nderiv=2>],
       [<AutoDiffXd 1.0 nderiv=2>]], dtype=object)

In [118]:
test = np.array([
    [1., 2.],
    [3., 4.]
])
type(test)

numpy.ndarray

In [125]:
autodiff_in = np.array(
    [AutoDiffXd(1.0, [1.0, 0.0]), AutoDiffXd(1.0, [1.0, 0.0])]
).reshape()
autodiff_in
type(autodiff_in), autodiff_in.shape

(numpy.ndarray, (2,))

In [129]:
q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
test = np.array([AutoDiffXd(x) for x in q])
test.shape

(7,)